# Ensemble classification model in earthquake building damage level detection

# Importing Libraries

In [1]:
!pip install mljar-supervised

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 112 kB 3.6 MB/s 
     |████████████████████████████████| 193.6 MB 50 kB/s 
     |████████████████████████████████| 2.0 MB 56.7 MB/s 
     |████████████████████████████████| 76.6 MB 1.5 MB/s 
     |████████████████████████████████| 72 kB 1.1 MB/s 
     |████████████████████████████████| 575 kB 43.4 MB/s 
     |████████████████████████████████| 72 kB 812 kB/s 
     |████████████████████████████████| 348 kB 101.9 MB/s 
     |████████████████████████████████| 209 kB 87.4 MB/s 
     |████████████████████████████████| 81 kB 12.1 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 147 kB 94.3 MB/s 
     |████████████████████████████████| 112 kB 93.3 MB/s 
  Created wheel for mljar-supervised: filename=mljar_supervised-0.11.3-py3-none-any.whl size=14665

In [2]:
import numpy as np
import pandas as pd
import cv2

from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
from google.colab import drive

import seaborn as sns
import matplotlib.pyplot as plt

# Data Loading

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
X = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Richter's Predictor: Modeling Earthquake Damage/data/train_values.csv", index_col='building_id')

In [5]:
X.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [6]:
Y= pd.read_csv("/content/drive/My Drive/Colab Notebooks/Richter's Predictor: Modeling Earthquake Damage/data/train_labels.csv", index_col='building_id')

In [7]:
Y.head()

,damage_grade
building_id,
802906,3
28830,2
94947,3
590882,2
201944,3


In [8]:
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Richter's Predictor: Modeling Earthquake Damage/data/test_values.csv", index_col='building_id')

In [9]:
X.describe()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
count,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,...,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000
mean,13.900353,701.074685,6257.876148,2.129723,26.535029,8.018051,5.434365,0.088645,0.761935,0.034332,...,0.064378,0.033626,0.008101,0.000940,0.000361,0.001071,0.000188,0.000146,0.000088,0.005119
std,8.033617,412.710734,3646.369645,0.727665,73.565937,4.392231,1.918418,0.284231,0.425900,0.182081,...,0.245426,0.180265,0.089638,0.030647,0.018989,0.032703,0.013711,0.012075,0.009394,0.071364
min,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000000,350.000000,3073.000000,2.000000,10.000000,5.000000,4.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,12.000000,702.000000,6270.000000,2.000000,15.000000,7.000000,5.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,21.000000,1050.000000,9412.000000,2.000000,30.000000,9.000000,6.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30.000000,1427.000000,12567.000000,9.000000,995.000000,100.000000,32.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Feature Engineering: Finding relation between geo_levels using seq_to_seq method (LSTM)

<font size=2> **Below is just an example to show how the geo_levels might be related in some way to each other.**
**For geo_level_1 = 6 and geo_level_2=487 we see there are a lot of same geo_levels-3 indicating that there is some relation between these levels.**</font>

In [10]:
pd.DataFrame(X['geo_level_3_id'][X['geo_level_2_id']==487][X['geo_level_1_id']==6]).head(50)

,geo_level_3_id
building_id,
802906,12198
959468,12198
410679,7439
423950,7439
215003,12198
594969,835
695349,2908
869506,7439
141511,382


### Converting the geo_levels 1 and 2 into one hot encoder with shared vocabulary

In [11]:
l1 = pd.concat([X['geo_level_1_id'], test['geo_level_1_id']])
l2 = pd.concat([X['geo_level_2_id'], test['geo_level_2_id']])
l3 = pd.concat([X['geo_level_3_id'], test['geo_level_3_id']])

In [12]:
#get shared vocab of l1 and l2
inp = pd.concat([l1,l2]).to_numpy().reshape(-1,1)

In [13]:
l1 = pd.concat([X['geo_level_1_id'], test['geo_level_1_id']]).to_numpy().reshape(-1,1)

In [14]:
l2 = pd.concat([X['geo_level_2_id'], test['geo_level_2_id']]).to_numpy().reshape(-1,1)

In [15]:
onehot = OneHotEncoder(sparse=False)

In [16]:
onehot.fit(inp)

OneHotEncoder(sparse=False)

In [17]:
inp.shape

(694938, 1)

In [18]:
onehot.categories_

[array([   0,    1,    2, ..., 1425, 1426, 1427])]

In [19]:
l1_hot = onehot.transform(l1)

In [20]:
l1_hot.shape

(347469, 1419)

In [21]:
l2_hot = onehot.transform(l2)

In [22]:
l2_hot.shape

(347469, 1419)

In [23]:
fin_inp = np.stack((l1_hot,l2_hot), axis=1)
fin_inp.shape

(347469, 2, 1419)

### Converting geo_level_3 into onehot

In [24]:
l3_hot = np.array(pd.get_dummies(l3))

In [25]:
l3_hot.shape

(347469, 11861)

### Creating a LSTM model to get relation features.

In [26]:
inpx = Input( shape=fin_inp.shape[1:] )  

lstm = LSTM(16, )(inpx)

lvl3 = Dense(l3_hot.shape[1], activation='sigmoid')(lstm)

modelf = Model(inputs= inpx, outputs=lvl3)

modelf.compile(loss='binary_crossentropy', optimizer='adam')

modelf.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2, 1419)]         0         
                                                                 
 lstm (LSTM)                 (None, 16)                91904     
                                                                 
 dense (Dense)               (None, 11861)             201637    
                                                                 
Total params: 293,541
Trainable params: 293,541
Non-trainable params: 0
_________________________________________________________________


In [27]:
modelf.fit(fin_inp, l3_hot, epochs=30, batch_size=128)

Epoch 1/30
2715/2715 [==============================] - 19s 5ms/step - loss: 0.0396
Epoch 2/30
2715/2715 [==============================] - 12s 5ms/step - loss: 9.7171e-04
Epoch 3/30
2715/2715 [==============================] - 12s 5ms/step - loss: 8.4860e-04
Epoch 4/30
2715/2715 [==============================] - 13s 5ms/step - loss: 8.3367e-04
Epoch 5/30
2715/2715 [==============================] - 12s 5ms/step - loss: 8.3094e-04
Epoch 6/30
2715/2715 [==============================] - 12s 5ms/step - loss: 8.3015e-04
Epoch 7/30
2715/2715 [==============================] - 12s 5ms/step - loss: 8.2986e-04
Epoch 8/30
2715/2715 [==============================] - 12s 4ms/step - loss: 8.2971e-04
Epoch 9/30
2715/2715 [==============================] - 12s 5ms/step - loss: 8.2964e-04
Epoch 10/30
2715/2715 [==============================] - 12s 4ms/step - loss: 8.2958e-04
Epoch 11/30
2715/2715 [==============================] - 12s 5ms/step - loss: 8.2948e-04
Epoch 12/30
2715/2715 [===========

### Getting the relation vector from the output of intermediate layer

In [28]:
inter = Model(inputs=modelf.input, outputs=modelf.layers[1].output)

<font size=3>**Train**</font>

In [29]:
l1 = X['geo_level_1_id'].to_numpy().reshape(-1,1)

l2 = X['geo_level_2_id'].to_numpy().reshape(-1,1)

In [30]:
l1_hot = onehot.transform(l1)
l2_hot = onehot.transform(l2)


In [31]:
fin_inp = np.stack((l1_hot,l2_hot), axis=1)
fin_inp.shape

(260601, 2, 1419)

In [32]:
geo_fea_train = inter.predict(fin_inp)

8144/8144 [==============================] - 12s 1ms/step


<font size=3>**Test**</font>

In [33]:
l1 = test['geo_level_1_id'].to_numpy().reshape(-1,1)

l2 = test['geo_level_2_id'].to_numpy().reshape(-1,1)

In [34]:
l1_hot = onehot.transform(l1)
l2_hot = onehot.transform(l2)


In [35]:
fin_inp = np.stack((l1_hot,l2_hot), axis=1)
fin_inp.shape

(86868, 2, 1419)

In [36]:
geo_fea_test = inter.predict(fin_inp)

2715/2715 [==============================] - 4s 1ms/step


# Final  Data

In [37]:
print("train {}, test {}  ".format(geo_fea_train.shape, geo_fea_test.shape))

train (260601, 16), test (86868, 16)  


### Preprocessing the data with pd.get_dummies and adding the relation features

In [38]:
fx_tr = np.hstack((np.array(pd.get_dummies(X.copy().drop(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'], axis=1))),
                   geo_fea_train))

In [39]:
fx_tr

array([[ 2.        , 30.        ,  6.        , ..., -0.96320885,
         0.96278083,  0.96329087],
       [ 2.        , 10.        ,  8.        , ..., -0.9631815 ,
         0.96186888,  0.96308053],
       [ 2.        , 10.        ,  5.        , ..., -0.96279722,
         0.77019542,  0.96302754],
       ...,
       [ 3.        , 55.        ,  6.        , ..., -0.96302277,
         0.96220833,  0.96299958],
       [ 2.        , 10.        , 14.        , ..., -0.96365595,
         0.96350259,  0.9636119 ],
       [ 3.        , 10.        ,  7.        , ..., -0.96280432,
         0.77021807,  0.96302682]])

In [40]:
fx_tr.shape

(260601, 81)

In [41]:
test.fillna(value=0, inplace=True)

In [42]:
fx_te = np.hstack((np.array(pd.get_dummies(test.copy().drop(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'], axis=1))), geo_fea_test))

fx_te.shape

(86868, 81)

In [43]:
fy_tr = Y.to_numpy().ravel()
fy_tr.shape

(260601,)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(fx_tr, fy_tr, test_size=0.2, random_state=42)

# Applying machine learning

In [53]:
automl = AutoML(
    mode="Compete",
    algorithms=['Random Forest','Extra Trees','LightGBM','Xgboost','CatBoost'],
    model_time_limit=30*60,
    start_random_models=10,
    hill_climbing_steps=3,
    top_models_to_improve=3,
    golden_features=True,
    features_selection=False,
    stack_models=True,
    train_ensemble=True,
    explain_level=1,
    results_path="/content/drive/My Drive/Colab Notebooks/Richter's Predictor: Modeling Earthquake Damage/output/models/mljar_3",
    eval_metric="f1",
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 10,
        "shuffle": True,
        "stratify": True,
        "random_seed": 42
    }
)

In [55]:
automl.fit(X_train, y_train)

AutoML directory: /content/drive/My Drive/Colab Notebooks/Richter's Predictor: Modeling Earthquake Damage/output/models/mljar_3
The task is multiclass_classification with evaluation metric f1
AutoML will use algorithms: ['Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'hill_climbing_1', 'hill_climbing_2', 'hill_climbing_3', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 5 models
1_Default_LightGBM f1 0.73655 trained in 984.09 seconds


`feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
`feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
`feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
`feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
`feval` is deprecated, use `custom_metric` instead.  They have different beh

2_Default_Xgboost f1 0.734704 trained in 1007.55 seconds
3_Default_CatBoost f1 0.728804 trained in 276.3 seconds
4_Default_RandomForest f1 0.654988 trained in 233.45 seconds
5_Default_ExtraTrees f1 0.655027 trained in 265.36 seconds
Skip not_so_random because of the time limit.
Skip golden_features because of the time limit.
Skip kmeans_features because of the time limit.
Skip hill_climbing_1 because of the time limit.
Skip hill_climbing_2 because of the time limit.
* Step hill_climbing_3 will try to check up to 8 models
6_LightGBM f1 0.737044 trained in 721.61 seconds


`feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
`feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
`feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
`feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
`feval` is deprecated, use `custom_metric` instead.  They have different beh

7_Xgboost f1 0.732003 trained in 1159.12 seconds


`feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
`feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
`feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
`feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
`feval` is deprecated, use `custom_metric` instead.  They have different beh

8_Xgboost f1 0.736234 trained in 881.52 seconds
9_CatBoost f1 0.728142 trained in 295.23 seconds
10_ExtraTrees f1 0.655027 trained in 259.62 seconds
11_ExtraTrees f1 0.655027 trained in 261.99 seconds
12_RandomForest f1 0.654988 trained in 231.1 seconds
13_RandomForest f1 0.654988 trained in 231.94 seconds
Skip boost_on_errors because of the time limit.
* Step ensemble will try to check up to 1 model
Ensemble f1 0.737169 trained in 22.08 seconds
Skip stack because no parameters were generated.
Skip ensemble_stacked because no parameters were generated.
AutoML fit time: 6887.17 seconds
AutoML best model: Ensemble


AutoML(algorithms=['Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost',
                   'CatBoost'],
       eval_metric='f1', explain_level=1, features_selection=False,
       golden_features=True, hill_climbing_steps=3, mode='Compete',
       model_time_limit=1800,
       results_path="/content/drive/My Drive/Colab Notebooks/Richter's "
                    'Predictor: Modeling Earthquake '
                    'Damage/output/models/mljar_3',
       stack_models=True, start_random_models=10, top_models_to_improve=3,
       validation_strategy={'k_folds': 10, 'random_seed': 42, 'shuffle': True,
                            'stratify': True, 'validation_type': 'kfold'})

# Submit

In [56]:
pred_votting_base_test = automl.predict(fx_te)

sub_csv = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Richter's Predictor: Modeling Earthquake Damage/data/submission_format.csv")

sub_csv["damage_grade"] = pred_votting_base_test
sub_csv.to_csv("/content/drive/My Drive/Colab Notebooks/Richter's Predictor: Modeling Earthquake Damage/output/submit/mljar_3.csv", index=False) #0.7367